In [5]:
import osmnx as ox
import geopandas as gpd
import pandas as pd 
import numpy as np


class World(object):
    
    def __init__(self, geofile_name='datafiles/Buildings_Gangelt_MA_3.csv', from_file=False, number_of_locs=100):
        self.from_file = from_file
        self.geofile_name = geofile_name
        self.number_of_locs = number_of_locs
        
        
        if self.from_file:
            self.locations = self.initialize_locs_from_file()
        else:    
            self.locations = self.initialize_locs_random()
            
        self.neighbourhoods = self.initialize_neighbourhoods()    
            
            

    def initialize_locs_random(self):# orginal
        locations = {}
        for n in range(self.number_of_locs):
            locations[n] = (Location(n, (n, 0), 'dummy_loc',1,1e-7))

        locations[3] = Location(3, (3, 0), 'hospital',1,1e-7)
        locations[0] = Location(0, (0, 0), 'hospital',1,1e-7)
        return locations
    
    def initialize_locs_from_file(self):
        locations = {}
        self.df_buildings = pd.read_csv(self.geofile_name)
        
        #work_place = list(self.df_buildings['amenity'].unique())\
        #        +list(self.df_buildings['shop'].unique())
        #healthcare = ['baby_hatch',
         #            'clinic',
         #            'dentist',
         #            'doctors',
         #            'hospital',
          #           'nursing_home',
          #           'pharmacy',
          #           'social_facility']
        #for x in healthcare:
        #    while x in work_place: work_place.remove(x)    
        #col_names=['amenity','shop']

        
        
        for i,x in enumerate(self.df_buildings.index):
            row = self.df_buildings.loc[x]
            try:
                if np.isnan(row['amenity']):
                    building_type = 'residence'
                else:
                    building_type = row['amenity']
            except:
                building_type = row['amenity']

            #building_type = location_settings(row[col_names],work_place, healthcare)  
            locations[i] = Location(x, (row['building_coordinates_x'],row['building_coordinates_y']),
                                    building_type,
                                    row['neighbourhood'],
                                    row['building_area'],)
        return locations
    
    
    def location_settings(building_lst, workplace:list, healthcare:list):
        building_type = 'Residence'
        if any(elem in healthcare for elem in building_lst):
            building_type = 'Healthcare'
        elif any(elem in workplace for elem in building_lst):
            building_type = 'Economy'
        return building_type
    
    def initialize_neighbourhoods(self):
        neighbourhoods = {1: Neighbourhood(self.locations)}
        return neighbourhoods
    
    
    
    
class Location(object):
    def __init__(self, ID, coordinates, location_type, neighbourhood, area, people_present=set(), location_factor=0.001): # runs good with 50 people and 10 infected and 5 location, add Neighbouhood_ID
        self.ID = ID
        self.people_present = people_present
        self.location_factor = location_factor
        self.coordinates = coordinates # () tuples
        self.location_type = location_type # add 'hospital'
        self.neighbourhood_ID = neighbourhood 
        self.distances = {}
        self.ids_of_location_types = {} # loc_id : distance 
        
class Neighbourhood(object):
    def __init__(self, locations):
        self.locations = locations
        self.proximity_matrix = self.calculate_proximity_matrix()
        self.ID = 1  # todo

    def calculate_proximity_matrix(self):  # create distances
        matrix = np.zeros((len(list(self.locations)), len(list(self.locations))))  # create

        for i, x in enumerate(self.locations.values()):
            ids = []
            types = []
            for k, y in enumerate(self.locations.values()):
                ids.append(y.ID)
                types.append(y.location_type)
                matrix[i, k] = np.sqrt(
                    (x.coordinates[0] - y.coordinates[0]) ** 2 + (x.coordinates[1] - y.coordinates[1]) ** 2)
            x.distances = dict(zip(ids, list(matrix[i, :])))
            location_types_in_neighbourhood = dict(zip(ids, types))
            ids_of_location_types = {}

            for t in list(set(list(location_types_in_neighbourhood.values()))):
                l = [x for x in location_types_in_neighbourhood if location_types_in_neighbourhood[x] == t]
                ids_of_location_types[t] = l
            x.ids_of_location_types = ids_of_location_types

        return matrix        


In [6]:
w = World(from_file=True)

In [ ]:
w.neighbourhoods

In [ ]:
for i in range (400):
    print(w.locations[10].distances)

In [ ]:
df_buildings = pd.read_csv('datafiles/Buildings_Gangelt_MA_3.csv')
gpd.read_file('datafiles/Buildings_Gangelt_MA_3.geojson')['amenity'].unique()

In [ ]:
for x in df_buildings['amenity']:
    try:
        if np.isnan(x):
            print('residence')
    except:
        print(x)

In [ ]:
world=World()

In [ ]:
def location_settings(building_lst, workplace:list, healthcare:list):
    building_type = 'Residence'
    if any(elem in healthcare for elem in building_lst):
        building_type = 'Healthcare'
    elif any(elem in workplace for elem in building_lst):
        building_type = 'Economy'
    return building_type
#r1 = gpd.read_file('datafiles/Buildings_Heinsberg.geojson')
def initialize_locs(self):
    locations = np.empty(len(df_buildings.index),dtype='object_')
    
    df_buildings = pd.read_csv('datafiles/Buildings_Gangelt_MA_3.csv')
    #Id coordinates type
    #Location()
    work_place = list(df_buildings['amenity'].unique())\
                +list(df_buildings['shop'].unique())
    healthcare = ['baby_hatch',
                 'clinic',
                 'dentist',
                 'doctors',
                 'hospital',
                 'nursing_home',
                 'pharmacy',
                 'social_facility']
    for x in healthcare:
        while x in work_place: work_place.remove(x)    
    col_names=['amenity','shop']
    

    for i,x in enumerate(df_buildings.index):
        row = df_buildings.loc[x]
        building_type = location_settings(row[col_names],work_place, healthcare)  
        
        locations[i] = Location(x, (row['building_coordinates_x'],row['building_coordinates_y']),building_type )
    return locations

In [ ]:
for loc in locations:
    if loc.location_type == 'Healthcare':
        print('health')

In [ ]:

for x in df_buildings.index:
    row = df_buildings.loc[x]
    print()